> 50. Pow(x, n)

In [ ]:
class Solution:
    def myPow(self, x: float, n: int) -> float:
        # use a for loop over n and each time multiply x? --> brute force
        # use the fact that x^(2n) = x^n * x^n with recursion
        if not n:
            return 1
        
        if n < 0:
            return float(1 / self.myPow(x, -n))
        
        if n % 2 == 0:
            return self.myPow(x * x, n//2)
        
        else:
            return self.myPow(x * x, n//2) * x

> 973. K Closest Points to Origin

In [ ]:
class Solution:
    def kClosest(self, points: List[List[int]], k: int) -> List[List[int]]:
        # can take the squard value of the distance to make it easier for calculations
        # first method, sorting (time = nlogn, space = n)
        """
        points.sort(key=self.squaredDistance)
        return points[:k]
        """
        
        # second method, use a max heap (time = nlog(k), space = k)
        """
        # use negative values of the distance to heapify
        # save the index of the points in the heap as well
        heap = [(-self.squaredDistance(points[i]),i) for i in range(k)]
        # heapify heap to sort values
        heapq.heapify(heap)
        # loop through remaining values, if the value is larger (smaller with negative)
        # we will insert into heap and push out the smallest value if the value is 
        # less than the lowest value
        for i in range(k, len(points)):
            dist = -self.squaredDistance(points[i])
            if dist > heap[0][0]:
                heapq.heappushpop(heap, (dist, i))
                
        return [points[i] for (_, i) in heap]
        """
        
        # 3rd method, use a modified binary search (time = n (average), space = n)
        # create a target distance and see how many points fall inside it and 
        # its relative amount to k, update or remove points based on if the
        # number pof points is lower or higher than k
        
        """
        # create a list of distances 
        distances = [self.squaredDistance(p) for p in points]
        # create a reference list
        remaining = [i for i in range(len(points))]
        
        # create a min and max target
        minDist = 0
        maxDist = max(distances)
        
        # binary search but iterate over k
        closest = []
        while k:
            mid = (minDist + maxDist) // 2
            closer, further = self.binarySplitHelper(mid, remaining, distances)
            if len(closer) > k:
                remaining = closer
                maxDist = mid
            else:
                closest.extend(closer)
                k -= len(closer)
                remaining = further
                minDist = mid
                
        return [points[i] for i in closest]
        """
        # 4th method, use quick select (time = n (average), space = c)
        # quick select works by selecting a pivot and iterating through the elements in a list
        # so the elements below the pivot will be below the pivot value and vice versa
        
        return self.quickSelect(points, k)
    
    def quickSelect(self, points, k):
        l = 0
        r = len(points) - 1
        pivotIndex = r + 1 # make sure the pivot index is always not equal to k for initial pass
        
        while pivotIndex != k:
            pivotIndex = self.partition(l, r, points)
            if pivotIndex < k:
                l = pivotIndex
            else:
                r = pivotIndex - 1
        
        return points[:k]
    
    def partition(self, l, r, points):
        pivot = (l + r) // 2
        pivotDist = self.squaredDistance(points[pivot])
        while l < r:
            if self.squaredDistance(points[l]) >= pivotDist:
                points[l], points[r] = points[r], points[l]
                r -= 1
            else:
                l += 1
        # need the left pointer to be just past the end of the the left range 
        # so we can return it as the new pivot index
        if self.squaredDistance(points[l]) < pivotDist:
            l += 1
        return l
        
            
    def binarySplitHelper(self, mid, remaining, distances):
        # holds elements less than the mid point
        closer = []
        # holds elements further than the mid point
        further = []
        # remember to iterate over remaining since it is a list of indicies
        for i in remaining:
            if distances[i] <= mid:
                closer.append(i)
            else:
                further.append(i)
                
        return closer, further
        
    def squaredDistance(self, point):
        return point[0] ** 2 + point[1] ** 2

> 670. Maximum Swap

In [ ]:
# Basic idea:
# Find a index i, where there is a increasing order
# On the right side of i, find the max value (max_val) and its index (max_idx)
# On the left side of i, find the most left value and its index (left_idx), which is less than max_val
# Swap above left_idx and max_idx if necessary
# Please check the comments for more detail

class Solution:
    def maximumSwap(self, num: int) -> int:
        # convert integer into an array of strings
        arrNum = [n for n in str(num)]
        # use a placeholder for the ascending index of the array (if there is one)
        ascIndex = -1
        
        # iterate over the array and check if there is a point where the value at the 
        # subsequent index is greater than the current index, if so, we know a swap could increase
        # the value of the integer
        for i in range(len(arrNum) - 1):
            if int(arrNum[i]) < int(arrNum[i + 1]) and ascIndex == -1:
                # once the above condition is met, record the index
                ascIndex = i
                break 
                
        # if all the values are in descending order, the string is at a max value
        if ascIndex == -1:
            return num
        
        # create a left and right array that contains values until the inflection point (ascIndex + 1)
        # the right array will contain a value to be swapped
        leftAsc = arrNum[:ascIndex + 1]
        rightAsc = arrNum[ascIndex + 1:]
        
        # iterate over the right array to find the rightmost max value and the corresponding index
        # tricky part here, we need to find the rightmost max value
        # i.e. if there is a tie in max value, pick the right most index using the >= operator
        maxAsc = 0
        maxInd = None
        for i, n in enumerate(rightAsc):
            if int(n) >= maxAsc:
                # find the max value and its respectice indice
                maxAsc = int(n)
                maxInd = i + len(leftAsc)
        # iterate over the left array to find the element to swap
        # i.e. first element in the array where the value is less then the max value from the right array
        for i, n in enumerate(leftAsc):
            if int(n) < maxAsc:
                arrNum[i], arrNum[maxInd] = arrNum[maxInd], arrNum[i]
                break
        
        # return the swapped array in integer form
        return int("".join(arrNum))